# Description

TODO

# Modules loading

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
from pathlib import Path

import numpy as np
import pandas as pd
from IPython.display import HTML
from tqdm import tqdm

from entity import Trait, Gene

import conf

# Settings

In [3]:
EXPERIMENT_NAME = "lv"
LIPIDS_GENE_SET = "gene_set_decrease"

In [4]:
RANDOM_SEED = 0
N_PERMUTATIONS = 1000
N_TOP_TRAITS = 25

In [5]:
OUTPUT_DIR = Path(
    conf.RESULTS["CRISPR_ANALYSES"]["BASE_DIR"],
    f"{EXPERIMENT_NAME}-{LIPIDS_GENE_SET}",
    "permutations",
)
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)
display(OUTPUT_DIR)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/crispr_analyses/lv-gene_set_decrease/permutations')

In [6]:
lipids_related_traits = {
    "celiac disease",
    "4079_raw-Diastolic_blood_pressure_automated_reading",
    "6150_100-Vascularheart_problems_diagnosed_by_doctor_None_of_the_above",
    "I9_UAP-Unstable_angina_pectoris",
    "6150_4-Vascularheart_problems_diagnosed_by_doctor_High_blood_pressure",
    "malabsorption syndrome",
    "K11_COELIAC-Coeliac_disease",
    "hypertension",
    "atherosclerosis",
    "4080_raw-Systolic_blood_pressure_automated_reading",
}

# Data loading

## PhenomeXcan data (S-MultiXcan)

In [7]:
smultixcan_results_filename = conf.PHENOMEXCAN[
    "SMULTIXCAN_EFO_PARTIAL_MASHR_ZSCORES_FILE"
]

display(smultixcan_results_filename)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/data/phenomexcan/gene_assoc/smultixcan-efo_partial-mashr-zscores.pkl')

In [8]:
results_filename_stem = smultixcan_results_filename.stem
display(results_filename_stem)

'smultixcan-efo_partial-mashr-zscores'

In [9]:
smultixcan_results = pd.read_pickle(smultixcan_results_filename)

In [10]:
smultixcan_results.shape

(22515, 3752)

In [11]:
smultixcan_results.head()

,100001_raw-Food_weight,100002_raw-Energy,100003_raw-Protein,100004_raw-Fat,100005_raw-Carbohydrate,100006_raw-Saturated_fat,100007_raw-Polyunsaturated_fat,100008_raw-Total_sugars,100009_raw-Englyst_dietary_fibre,100010-Portion_size,...,visual impairment,vitiligo,vitreous body disease,vocal cord polyp,voice disorders,wellbeing measurement AND family relationship,wheezing,whooping cough,worry measurement,wrist fracture
gene_name,,,,,,,,,,,,,,,,,,,,,
ENSG00000000419,1.145442,0.724557,0.090876,0.298165,1.134347,1.371138,0.065718,0.794317,0.600342,0.317652,...,0.360518,1.351624,1.157695,0.835289,1.173072,1.337280,1.743822,1.017226,1.512184,0.972241
ENSG00000000457,0.618066,1.028131,2.218420,0.762584,0.934418,0.192993,1.080230,0.765997,0.375898,0.678731,...,2.134504,0.127830,0.534690,0.120516,0.517464,2.545363,0.673331,2.003092,0.344000,2.033122
ENSG00000000460,0.515724,0.403596,1.251359,0.433091,0.413466,0.246261,1.236151,0.827430,0.571985,0.782174,...,1.768905,0.992408,0.548215,0.412341,1.499415,1.366780,0.443318,0.417630,0.225934,1.613246
ENSG00000000938,0.280781,0.253910,0.879148,0.352705,0.051846,0.184212,0.148566,0.009989,0.363751,0.374514,...,0.656552,2.046041,2.746832,0.108211,1.008258,0.755695,0.896228,0.875047,0.476405,1.693057
ENSG00000000971,0.548127,0.389877,0.723469,1.167250,0.315952,0.324939,1.613932,0.311432,0.333548,1.807243,...,0.260482,0.646204,1.080240,0.678330,1.465358,0.307672,0.118376,1.419812,0.000002,1.040737


### Gene IDs to Gene names

In [12]:
smultixcan_results = smultixcan_results.rename(index=Gene.GENE_ID_TO_NAME_MAP)

In [13]:
smultixcan_results.shape

(22515, 3752)

In [14]:
smultixcan_results.head()

,100001_raw-Food_weight,100002_raw-Energy,100003_raw-Protein,100004_raw-Fat,100005_raw-Carbohydrate,100006_raw-Saturated_fat,100007_raw-Polyunsaturated_fat,100008_raw-Total_sugars,100009_raw-Englyst_dietary_fibre,100010-Portion_size,...,visual impairment,vitiligo,vitreous body disease,vocal cord polyp,voice disorders,wellbeing measurement AND family relationship,wheezing,whooping cough,worry measurement,wrist fracture
gene_name,,,,,,,,,,,,,,,,,,,,,
DPM1,1.145442,0.724557,0.090876,0.298165,1.134347,1.371138,0.065718,0.794317,0.600342,0.317652,...,0.360518,1.351624,1.157695,0.835289,1.173072,1.337280,1.743822,1.017226,1.512184,0.972241
SCYL3,0.618066,1.028131,2.218420,0.762584,0.934418,0.192993,1.080230,0.765997,0.375898,0.678731,...,2.134504,0.127830,0.534690,0.120516,0.517464,2.545363,0.673331,2.003092,0.344000,2.033122
C1orf112,0.515724,0.403596,1.251359,0.433091,0.413466,0.246261,1.236151,0.827430,0.571985,0.782174,...,1.768905,0.992408,0.548215,0.412341,1.499415,1.366780,0.443318,0.417630,0.225934,1.613246
FGR,0.280781,0.253910,0.879148,0.352705,0.051846,0.184212,0.148566,0.009989,0.363751,0.374514,...,0.656552,2.046041,2.746832,0.108211,1.008258,0.755695,0.896228,0.875047,0.476405,1.693057
CFH,0.548127,0.389877,0.723469,1.167250,0.315952,0.324939,1.613932,0.311432,0.333548,1.807243,...,0.260482,0.646204,1.080240,0.678330,1.465358,0.307672,0.118376,1.419812,0.000002,1.040737


### Remove duplicated gene entries

In [15]:
smultixcan_results.index[smultixcan_results.index.duplicated(keep="first")]

Index(['SPATA13', 'LINC01422', 'LINC00484', 'MAL2', 'GOLGA8M', 'LINC01115',
       'LYNX1'],
      dtype='object', name='gene_name')

In [16]:
smultixcan_results = smultixcan_results.loc[
    ~smultixcan_results.index.duplicated(keep="first")
]

In [17]:
smultixcan_results.shape

(22508, 3752)

### Some checks

In [18]:
# the data should have no NaN values
assert smultixcan_results.shape == smultixcan_results.dropna(how="any").shape

### Standardize S-MultiXcan results

In [19]:
_tmp = smultixcan_results.apply(lambda x: x / x.sum())

In [20]:
_tmp.shape

(22508, 3752)

In [21]:
assert _tmp.shape == smultixcan_results.shape

In [22]:
smultixcan_results = _tmp

In [23]:
smultixcan_results.head()

,100001_raw-Food_weight,100002_raw-Energy,100003_raw-Protein,100004_raw-Fat,100005_raw-Carbohydrate,100006_raw-Saturated_fat,100007_raw-Polyunsaturated_fat,100008_raw-Total_sugars,100009_raw-Englyst_dietary_fibre,100010-Portion_size,...,visual impairment,vitiligo,vitreous body disease,vocal cord polyp,voice disorders,wellbeing measurement AND family relationship,wheezing,whooping cough,worry measurement,wrist fracture
gene_name,,,,,,,,,,,,,,,,,,,,,
DPM1,0.000060,0.000039,0.000005,0.000016,0.000061,0.000075,0.000004,4.254814e-05,0.000031,0.000017,...,0.000020,0.000075,0.000064,0.000047,0.000066,0.000066,0.000069,0.000058,6.250232e-05,0.000034
SCYL3,0.000033,0.000055,0.000119,0.000041,0.000050,0.000011,0.000059,4.103114e-05,0.000020,0.000036,...,0.000118,0.000007,0.000030,0.000007,0.000029,0.000126,0.000027,0.000114,1.421837e-05,0.000070
C1orf112,0.000027,0.000022,0.000067,0.000023,0.000022,0.000014,0.000067,4.432186e-05,0.000030,0.000041,...,0.000098,0.000055,0.000030,0.000023,0.000084,0.000068,0.000018,0.000024,9.338414e-06,0.000056
FGR,0.000015,0.000014,0.000047,0.000019,0.000003,0.000010,0.000008,5.350929e-07,0.000019,0.000020,...,0.000036,0.000114,0.000152,0.000006,0.000057,0.000038,0.000035,0.000050,1.969101e-05,0.000058
CFH,0.000029,0.000021,0.000039,0.000063,0.000017,0.000018,0.000088,1.668206e-05,0.000017,0.000095,...,0.000014,0.000036,0.000060,0.000038,0.000083,0.000015,0.000005,0.000081,8.895497e-11,0.000036


## MultiPLIER Z matrix

In [24]:
multiplier_z = pd.read_pickle(conf.MULTIPLIER["MODEL_Z_MATRIX_FILE"])

In [25]:
multiplier_z.shape

(6750, 987)

In [26]:
multiplier_z.head()

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,...,LV978,LV979,LV980,LV981,LV982,LV983,LV984,LV985,LV986,LV987
GAS6,0.000000,0.0,0.039438,0.0,0.050476,0.000000,0.0,0.000000,0.590949,0.000000,...,0.050125,0.00000,0.033407,0.000000,0.000000,0.005963,0.347362,0.0,0.000000,0.000000
MMP14,0.000000,0.0,0.000000,0.0,0.070072,0.000000,0.0,0.004904,1.720179,2.423595,...,0.000000,0.00000,0.001007,0.000000,0.035747,0.000000,0.000000,0.0,0.014978,0.000000
DSP,0.000000,0.0,0.000000,0.0,0.000000,0.041697,0.0,0.005718,0.000000,0.000000,...,0.020853,0.00000,0.000000,0.000000,0.000000,0.005774,0.000000,0.0,0.000000,0.416405
MARCKSL1,0.305212,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.161843,0.149471,...,0.027134,0.05272,0.000000,0.030189,0.060884,0.000000,0.000000,0.0,0.000000,0.448480
SPARC,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.014014,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.067779,0.0,0.122417,0.062665


## PhenomeXcan projection

In [27]:
input_filepath = Path(
    conf.RESULTS["PROJECTIONS_DIR"],
    "projection-smultixcan-efo_partial-mashr-zscores.pkl",
).resolve()
display(input_filepath)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/projections/projection-smultixcan-efo_partial-mashr-zscores.pkl')

In [28]:
phenomexcan_data = pd.read_pickle(input_filepath).T

In [29]:
phenomexcan_data.shape

(3752, 987)

In [30]:
phenomexcan_data.head()

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,...,LV978,LV979,LV980,LV981,LV982,LV983,LV984,LV985,LV986,LV987
100001_raw-Food_weight,-0.021292,0.056061,0.001650,0.026265,-0.015616,0.040333,-0.012050,0.027985,0.021028,0.027118,...,0.033575,0.051033,-0.038899,0.084371,0.027174,-0.032639,0.015128,-0.018430,0.016253,-0.021666
100002_raw-Energy,-0.046815,-0.009864,-0.004262,-0.007235,0.010799,0.021540,0.008009,0.025354,0.009816,0.041230,...,0.006675,0.022421,-0.014042,0.002723,0.066739,-0.031207,-0.001040,0.050533,-0.007979,-0.017742
100003_raw-Protein,-0.021585,-0.028888,0.019211,0.004304,0.017792,0.023804,-0.001246,0.023492,0.007045,-0.002453,...,0.029808,0.030413,-0.065487,0.000143,0.001568,-0.027622,-0.006422,0.040084,-0.008175,-0.014154
100004_raw-Fat,-0.030324,-0.053573,0.007484,-0.042464,0.010591,0.031777,0.007824,0.018422,-0.027632,0.028237,...,0.017412,0.018589,0.003284,-0.004189,0.040693,-0.036318,0.019393,0.029495,-0.015966,-0.002160
100005_raw-Carbohydrate,-0.017773,0.006951,0.004548,-0.001093,0.001972,-0.005518,0.033477,0.017333,0.040532,0.029409,...,-0.007403,-0.000781,0.006414,0.009671,0.034106,-0.008757,0.022819,0.055545,0.005864,-0.055875


## LVs enrichment on DEG from CRISPR screen

In [31]:
deg_enrich = pd.read_csv(
    Path(conf.RESULTS["CRISPR_ANALYSES"]["BASE_DIR"], "fgsea-all_lvs.tsv").resolve(),
    sep="\t",
)

In [32]:
deg_enrich.shape

(1973, 11)

In [33]:
deg_enrich.head()

,pathway,pval,padj,log2err,ES,NES,size,leadingEdge,lv,rep_idx,fdr
0,gene_set_decrease,0.198801,0.251748,0.091680,0.578418,1.123767,35,"PTBP1, KEAP1, PEX14, DLST, PCYT2, MAD2L2, GLRX...",LV1,2,0.755621
1,gene_set_increase,0.251748,0.251748,0.078711,0.539155,1.078565,63,"CHERP, RANGAP1, HNRNPL, RPS2, E4F1, TAF1C, GAT...",LV1,2,0.779947
2,gene_set_decrease,0.963037,0.999001,0.008938,0.434799,0.678421,20,"VDR, ACVR1B, KEAP1, NDUFV2, PEX14, NDUFS3, WDR26",LV10,5,1.000000
3,gene_set_increase,1.000000,1.000000,0.000000,0.236619,0.378164,25,"USP39, SRP19, ZNF3, RPL18, RPS19, RAP1GDS1, CH...",LV10,2,1.000000
4,gene_set_decrease,0.042957,0.085914,0.216543,0.831205,1.711885,27,"NDUFB7, PTBP1, RRAGC, PPP2R2B, SQLE",LV100,6,0.718678


## MultiPLIER summary

In [34]:
multiplier_model_summary = pd.read_pickle(conf.MULTIPLIER["MODEL_SUMMARY_FILE"])

In [35]:
multiplier_model_summary.shape

(2157, 5)

In [36]:
multiplier_model_summary.head()

,pathway,LV index,AUC,p-value,FDR
1,KEGG_LYSINE_DEGRADATION,1,0.388059,0.866078,0.956005
2,REACTOME_MRNA_SPLICING,1,0.733057,0.000048,0.000582
3,MIPS_NOP56P_ASSOCIATED_PRE_RRNA_COMPLEX,1,0.680555,0.001628,0.011366
4,KEGG_DNA_REPLICATION,1,0.549473,0.312155,0.539951
5,PID_MYC_ACTIVPATHWAY,1,0.639303,0.021702,0.083739


# Save list of traits

In [37]:
output_filepath = OUTPUT_DIR / "lipid_traits_list.pkl"
display(output_filepath)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/crispr_analyses/lv-gene_set_decrease/permutations/lipid_traits_list.pkl')

In [38]:
with open(output_filepath, "wb") as handle:
    pickle.dump(lipids_related_traits, handle, protocol=pickle.DEFAULT_PROTOCOL)

# Get gene modules under analysis

In [39]:
df = deg_enrich[
    deg_enrich["pathway"].isin((LIPIDS_GENE_SET,)) & (deg_enrich["padj"] < 0.05)
].sort_values("padj", ascending=True)

In [40]:
df.shape

(24, 11)

In [41]:
df.head()

,pathway,pval,padj,log2err,ES,NES,size,leadingEdge,lv,rep_idx,fdr
1283,gene_set_decrease,2.605291e-07,5.210582e-07,0.674963,0.858542,1.636153,43,"NDUFA4, COX6A1, ATP5O, NDUFB10, COX5A, NDUFS3,...",LV678,3,0.000257
1199,gene_set_decrease,1.205740e-03,2.411481e-03,0.455060,0.753059,1.430708,35,"PEX14, COX17, NDUFS3, ATP5O, COX6A1, DLST, NDU...",LV64,2,0.339847
1769,gene_set_decrease,1.610715e-03,3.221430e-03,0.455060,0.807492,1.596196,25,"NDUFA4, PEX14, STX18, OGDH, COX6A1, COX17, PIK...",LV897,3,0.397243
1055,gene_set_decrease,4.275828e-03,8.551656e-03,0.407018,0.897219,1.742375,23,"CSK, MRPS12",LV575,4,0.718678
1413,gene_set_decrease,5.763040e-03,1.152608e-02,0.407018,0.936411,1.741728,19,"COX7C, NDUFS6, FLCN, COX5A, NDUFS2",LV736,3,0.718678


In [42]:
important_lvs = df["lv"].unique()

In [43]:
display(important_lvs.shape)
assert important_lvs.shape[0] == 24

(24,)

In [44]:
important_lvs

array(['LV678', 'LV64', 'LV897', 'LV575', 'LV736', 'LV157', 'LV421',
       'LV469', 'LV250', 'LV420', 'LV558', 'LV630', 'LV750', 'LV99',
       'LV676', 'LV775', 'LV131', 'LV199', 'LV896', 'LV783', 'LV467',
       'LV498', 'LV515', 'LV343'], dtype=object)

# How many genes with nonzero weight on average in these modules?

In [45]:
multiplier_z[df["lv"].values].apply(lambda x: x[x > 0].shape[0]).describe()

count      24.000000
mean     2959.708333
std       358.184854
min      2364.000000
25%      2706.750000
50%      2897.000000
75%      3148.250000
max      3695.000000
dtype: float64

# How likely is to get relevant traits at the top of these 24 modules?

In [46]:
from multiplier import MultiplierProjection

In [47]:
np.random.seed(RANDOM_SEED)

In [48]:
z = multiplier_z  # .copy()

permutation_results = []

for i in tqdm(range(N_PERMUTATIONS)):
    traits = []

    sub_z = z[important_lvs]  # .copy()

    # shuffle index
    sub_z_index = sub_z.index.tolist()
    np.random.shuffle(sub_z_index)
    sub_z.index = sub_z_index

    new_model_z = z.drop(columns=important_lvs).join(sub_z)
    new_model_z = new_model_z.loc[z.index.tolist(), z.columns.tolist()]
    assert new_model_z.shape == multiplier_z.shape

    mproj = MultiplierProjection()
    new_proj = mproj.transform(smultixcan_results, multiplier_model_z=new_model_z)
    new_proj = new_proj.T

    for lv in important_lvs:
        _tmp = new_proj[lv]
        _tmp = _tmp[_tmp > 0.0].sort_values(ascending=False)
        traits.append(_tmp)

    traits_df = (
        pd.concat(traits)
        .reset_index()
        .groupby("index")
        .sum()
        .sort_values(0, ascending=False)
        .reset_index()
    ).rename(columns={"index": "trait", 0: "value"})

    top_traits = traits_df.head(N_TOP_TRAITS)
    permutation_results.append(lipids_related_traits.intersection(top_traits["trait"]))

100%|██████████| 1000/1000 [48:46<00:00,  2.93s/it]


## Save

In [49]:
output_filepath = OUTPUT_DIR / "permutation_results.pkl"
display(output_filepath)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/crispr_analyses/lv-gene_set_decrease/permutations/permutation_results.pkl')

In [50]:
with open(output_filepath, "wb") as handle:
    pickle.dump(permutation_results, handle, protocol=pickle.DEFAULT_PROTOCOL)